In [1]:
import os
import cv2 
import numpy as np
import rasterio

from scipy.ndimage import zoom, rotate
from matplotlib import pyplot as plt 


In [2]:
# T34VFJ and T35VLD = Riga


In [3]:
files = {}

for f in os.listdir("/home/kristaps/Projs/bulbulis/data"):
    if ".jp2" in f:
        parts = f.split("_")
        if parts[0] not in files:
            files[parts[0]] = {}
        if parts[1] not in files[parts[0]]:
            files[parts[0]][parts[1]] = {}
        files[parts[0]][parts[1]][parts[2]] = f"/home/kristaps/Projs/bulbulis/data/{f}"


In [4]:
dates = []
for f in files.values():
    dates += list(f.keys())
dates = sorted(list(set(dates)))
    

In [5]:
deg = 5
v_offset = 300
w = 10980
h = 4*10980+v_offset


In [6]:
left = np.dstack((np.zeros((h, w)), np.zeros((h, w)), np.zeros((h, w))))
left_offsets = {
    "T34VFK": 0,
    "T34VFJ": 1,
    "T34VFH": 2,
    "T34UFG": 3,
}
right = np.dstack((np.zeros((h, w)), np.zeros((h, w)), np.zeros((h, w))))
right_offsets = {
    "T35VLE": 0,
    "T35VLD": 1,
    "T35VLC": 2,
    "T35ULB": 3,
}

merged = np.dstack((np.zeros((h, 2*w)), np.zeros((h, 2*w)), np.zeros((h, 2*w))))


In [ ]:
gain = 2

for d in dates[10:20]:
    for k,v in left_offsets.items():
        if d in files[k]:
            fj = files[k][d]
            try:
                scl = np.repeat(np.repeat(rasterio.open(fj["SCL"], driver="JP2OpenJPEG").read(1), 2, 0), 2, 1)
                c_red = np.clip(rasterio.open(fj["B04"], driver="JP2OpenJPEG").read(1)*gain/10000, 0, 1)
                c_green = np.clip(rasterio.open(fj["B03"], driver="JP2OpenJPEG").read(1)*gain/10000, 0, 1) 
                c_blue = np.clip(rasterio.open(fj["B02"], driver="JP2OpenJPEG").read(1)*gain/10000, 0, 1)
                
                print(d, k)
                cell_offset = v*w+v_offset
                mask = ((scl != 0) & (scl != 1) & (scl != 3) & (scl != 7) & (scl != 8) & (scl != 9) & (scl != 10))
    
                left[cell_offset:cell_offset+w, :, 0][mask] = c_red[mask]
                left[cell_offset:cell_offset+w, :, 1][mask] = c_green[mask]
                left[cell_offset:cell_offset+w, :, 2][mask] = c_blue[mask]
    
                merged[cell_offset:cell_offset+w, :w, :][mask] = left[cell_offset:cell_offset+w, :, :][mask]
            except BaseException as e:
                print(e)
    for k,v in right_offsets.items():
        if d in files[k]:
            fj = files[k][d]
            try:
                scl = np.repeat(np.repeat(rasterio.open(fj["SCL"], driver="JP2OpenJPEG").read(1), 2, 0), 2, 1)
                c_red = np.clip(rasterio.open(fj["B04"], driver="JP2OpenJPEG").read(1)*gain/10000, 0, 1)
                c_green = np.clip(rasterio.open(fj["B03"], driver="JP2OpenJPEG").read(1)*gain/10000, 0, 1) 
                c_blue = np.clip(rasterio.open(fj["B02"], driver="JP2OpenJPEG").read(1)*gain/10000, 0, 1)
    
                print(d, k)
                cell_offset = v*w
                mask = ((scl != 0) & (scl != 1) & (scl != 3) & (scl != 7) & (scl != 8) & (scl != 9) & (scl != 10))
                right[cell_offset:cell_offset+w, :, 0][mask] = c_red[mask]
                right[cell_offset:cell_offset+w, :, 1][mask] = c_green[mask]
                right[cell_offset:cell_offset+w, :, 2][mask] = c_blue[mask]

                temp_right = rotate(right, deg, reshape=False)
                temp_scl = np.zeros((h, w))
                temp_scl[cell_offset:cell_offset+w, :] = scl
                temp_scl = rotate(temp_scl, deg, reshape=False)
                
                mask = ((temp_scl != 0) & (temp_scl != 1) & (temp_scl != 3) & (temp_scl != 7) & (temp_scl != 8) & (temp_scl != 9) & (temp_scl != 10))
                
                s = w-4750
                merged[:, s:w+s, :][mask] = np.clip(temp_right[mask], 0, 1)
            except BaseException as e:
                print(e)
            

20150810T100016 T34VFK
20150810T100016 T34VFJ
20150811T093006 T34VFH
20150811T093006 T34UFG
20150811T093006 T35VLE
20150811T093006 T35VLD
20150811T093006 T35VLC
20150811T093006 T35ULB
20150814T094006 T34VFK
20150814T094006 T34VFJ
20150814T094006 T34VFH
20150814T094006 T34UFG
20150814T094006 T35VLE
20150814T094006 T35VLD
20150814T094006 T35VLC
20150814T094006 T35ULB
20150817T095016 T34VFK
20150817T095016 T34VFJ
20150817T095016 T34VFH
20150817T095016 T34UFG
20150817T095016 T35VLE
20150817T095016 T35VLD
20150817T095016 T35VLC
20150817T095016 T35ULB
20150820T100016 T34VFK
20150820T100016 T34VFJ
20150821T093006 T34VFH
20150821T093006 T34UFG
20150821T093006 T35VLE
20150821T093006 T35VLD
20150821T093006 T35VLC
20150821T093006 T35ULB
20150824T094006 T34VFK
20150824T094006 T34VFJ
20150824T094006 T34VFH
20150824T094006 T34UFG
20150824T094006 T35VLE
20150824T094006 T35VLD
20150824T094006 T35VLC
20150824T094006 T35ULB
20150827T095016 T34VFK
20150827T095016 T34VFJ
20150827T095016 T34VFH
20150827T09

In [ ]:
plt.imshow(left)

In [ ]:
plt.imshow(right)

In [ ]:
plt.imshow(merged)